# Translate text from Arabic to English using IBM Language Translator service

## References
* [Watson Developer Cloud Python SDK](https://github.com/watson-developer-cloud/python-sdk)
* [How to read Dictionary from File in Python?](https://www.geeksforgeeks.org/how-to-read-dictionary-from-file-in-python)
* [Watson Python SDK samples](https://github.com/watson-developer-cloud/python-sdk)
* [Language Translator API Docs - Translate](https://cloud.ibm.com/apidocs/language-translator#translate)
* [Logging HOWTO](https://docs.python.org/3/howto/logging.html)
* [Difference between map, applymap and apply methods in Pandas](https://stackoverflow.com/questions/19798153)

## Steps
* [Create an IBM Cloud account](ibm.biz/sacloud)
* Create an instance of the [IBM Language Translator](https://cloud.ibm.com/catalog/services/language-translator) service
* Get credentials from [IBM Cloud Resources](http://cloud.ibm.com/resources) page of your instance and save them in ```Settings.json```.
* Load settings from ```Settings.json```.

## Installs
First time only

In [43]:
# coding: utf-8
%pip install ibm_cloud_sdk_core ibm_watson openpyxl --upgrade

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Imports

In [30]:
import json														# For reading settings
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator	# To authenticate to IBM Cloud
import logging
from ibm_watson import LanguageTranslatorV3						# For translating text
import pandas as pd
from os import path

## Set defaults

In [9]:
logging.getLogger().setLevel(logging.INFO)

## Initialization
### Load the Settings file

In [8]:
# Read the settings file as a dictionary
with open('Settings.json') as file:
	settings = json.loads( file.read() )

### Initialize the dictionary
Static dictionary for translated texts.
* Read the dictionary file, or
* Initalize an empty dictionary

In [22]:
try:
	with open(settings["dictionary"], encoding="utf-8") as file:
		dictionary = json.loads( file.read() )
	logging.info(f"Loaded dictionary file '{settings['dictionary']}'.")
except (FileNotFoundError, ValueError):
	logging.error(f"Could not load the dictionary file: '{settings['dictionary']}'.")
	logging.info(f"Starting a new dictionary.")
	# Initialize an empty dictionary
	dictionary = {}

INFO:root:Loaded dictionary file 'Dictionary.json'.


### Connection to the service instance

In [11]:
language_translator = LanguageTranslatorV3(
    version='2018-05-01',
    authenticator=IAMAuthenticator( settings["apikey"] ) )
language_translator.set_service_url( settings["service_instance_url"] )

## Translation function
Should not translate the following:
* English-only text
* Empty texts

In [48]:
# Translates text from Arabic to English
def translate_to_English(text):
	# Cast to string and trim
	text = str(text).strip()

	# Check if it was translated previously
	if text in dictionary:
		logging.debug(f"'{text}' is already in the dictionary.")
		return dictionary[text]
	
	# If the text is empty, return the same text and don't store it in the dictionary
	if not text:
		return text
	
	# Don't translate English text or numbers
	if text.isascii():
		return text
	
	#response = language_translator.translate(text, model_id='en-ar')
	response = language_translator.translate(text, model_id='ar-en')

	# If request succeeded, print the response, else print the error message
	if( response.status_code ) == 200:
		result = response.get_result()["translations"][0]["translation"]

		logging.debug(f"'{text}' is translated to '{result}'.")
		# Save the result in the dictionary for later use
		dictionary[text] = result
	else:
		result = f"{response.status_code}: {response.get_result()}"
		logging.error( result )
	
	return result

In [23]:
# Read the Excel file that needs to be translated
df = pd.read_excel( settings["file"] )

In [24]:
# Columns that should not be translated
columns_no_translation = [
	'Timestamp',
	'Email Address',
    'الاسم',
	'الإيميل الجامعي',
	'1. مدى رضاك عن المنصة التدريبية',
	'2. سهولة التسجيل في المنصة التدريبية',
	'3. سهولة استخدام الأدوات المتاحة في المنصة التدريبية',
	'4. سهولة التواصل مع الدعم الفني',
	'5. إمكانية التواصل مع المدرب من خلال المنصة',
	'6. مراعاة بيئة التعلم لاحتياجات جميع المتعلمين مع اختلاف قدراتهم واحتياجاتهم واهتماماتهم',
	'7. سهولة التنقل في البيئة التعليمية',
	'وضوح محتوى البرنامج التدريبي',
	'مناسبة المحتوى لمعارف وخبرات المتدرب',
	' تسلسل المادة العلمية',
	'تحقيق البرنامج التدريبي لتوقعاتك     ',
	' وضوح أهداف البرنامج التدريبي',
	'ﻋﺮض اﻟﻤﺤﺘﻮى ﺑﻄﺮق وأﻧﺸﻄﺔ ﺗﻌﻠﻴﻤﻴﺔ ﻣﺘﻨﻮﻋﺔ لضمان تقديمه بطريقة تسهل على المتعلم التعلم بفعالية',
	'تنوع المحتوى الرقمي (نصوص، مواد مسموعة، مواد مرئية)',
	'قدرة المدرب على إيصال وتسهيل المعلومة',
	'مستوى تمكن المدرب من محتوى البرنامج التدريبي',
	'التزام المدرب بزمن التدريب',
	'تقديم التغذية الراجعة من قبل المدرب',
	'وقت التدريب يومياً',
	'مناسبة المدة الزمنية المخصصة  للتدريب مع المحتوى التدريبي',
	'مناسبة تاريخ انطلاق البرنامج']

Translate data:

In [25]:
for column in df.columns:
	if column not in columns_no_translation:
		logging.debug(f"Translating the column '{column}'.")
		df[column] = df[column].apply(translate_to_English)

logging.debug(f"Dictionary has {len(dictionary)} entries.")

INFO:root:Translating the column 'الجنس'.
INFO:root:Translating the column 'الجامعة'.
INFO:root:Translating the column 'تسلسل المادة العلمية'.
INFO:root:Translating the column 'تحقيق البرنامج التدريبي لتوقعاتك'.
INFO:root:Translating the column 'وضوح أهداف البرنامج التدريبي'.
INFO:root:Translating the column 'ما هي أبرز الصعوبات التي واجهتك خلال التدريب على البرنامج بشكل عام من ناحية (المنصة - المحتوى التدريبي - المدرب)؟ '.
INFO:root:Translating the column ' هل لدريك مقترحات تطويرية لمكونات البرنامج التدريبي من ناحية (المنصة - المحتوى التدريبي - المدرب)؟ '.
INFO:root:Translating the column 'ماهي نقاط القوة في البرنامج؟'.
INFO:root:'—-' is translated to '--'.
INFO:root:Translating the column 'هل توصي زملاءك بالاشتراك في البرنامج مستقبلًا؟'.
INFO:root:Translating the column 'الوقت الأنسب لكم لتدريب مجموعات أخرى [الرغبة الأولى]'.
INFO:root:Translating the column 'الوقت الأنسب لكم لتدريب مجموعات أخرى [الرغبة الثانية]'.
INFO:root:Translating the column 'الوقت الأنسب لكم لتدريب مجموعات أخرى 

## Translate column names

In [47]:
# Rename columns of the Excel sheet
df.rename(mapper=translate_to_English, axis="columns", inplace=True)
logging.info(df.columns)

INFO:root:'الإيميل الجامعي' is translated to 'The university e-mail.'.
INFO:root:'1. مدى رضاك عن المنصة التدريبية' is translated to '1. How satisfied you are the training platform'.
INFO:root:'2. سهولة التسجيل في المنصة التدريبية' is translated to '2. Easy to register on the training platform'.
INFO:root:'3. سهولة استخدام الأدوات المتاحة في المنصة التدريبية' is translated to '3. Easy use of tools available on the training platform'.
INFO:root:'4. سهولة التواصل مع الدعم الفني' is translated to '4. Easy to communicate with technical support'.
INFO:root:'5. إمكانية التواصل مع المدرب من خلال المنصة' is translated to '5. The possibility of communicating with the coach through the platform'.
INFO:root:'6. مراعاة بيئة التعلم لاحتياجات جميع المتعلمين مع اختلاف قدراتهم واحتياجاتهم واهتماماتهم' is translated to '6. Take into account the learning environment for the needs of all learners with different abilities, needs and interests'.
INFO:root:'7. سهولة التنقل في البيئة التعليمية' is translated 

## Save the dictionary

In [35]:
# Write the dictionary file
with open(settings["dictionary"], mode="w", encoding="utf-8") as file:
	file.write( json.dumps(dictionary, ensure_ascii=False, indent=4) )
	
	logging.info(f"Dictionary saved to '{settings['dictionary']}'")

INFO:root:Dictionary saved to 'Dictionary.json'


## Save the translation

In [53]:
# Write the translated file in the same directory
file = path.join( path.dirname(settings["file"]), "Translated.xlsx" )
df.to_excel(file, index=False, freeze_panes=(1,6))
logging.info(f"Translation saved to '{file}'.")

INFO:root:Translation saved to 'C:\Translated.xlsx'.
